# 1. Import Libraries

In [1]:
import numpy as np

import pandas as pd

import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
	OneHotEncoder,
	OrdinalEncoder,
	StandardScaler,
	MinMaxScaler,
	PowerTransformer,
	FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import (
	RareLabelEncoder,
	MeanEncoder,
	CountFrequencyEncoder
)

import matplotlib.pyplot as plt

import warnings

c:\Users\lenovo i3\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## 2. display Settings

In [2]:
pd.set_option("display.max_columns", None)
sklearn.set_config(transform_output="pandas")
warnings.filterwarnings("ignore")

## 3. Read the Data

In [3]:
path =r"C:\Users\lenovo i3\OneDrive\Documents\Project Data\Compus x project\filght-price-prediction\dataset\train.csv"
train = pd.read_csv(path)

In [4]:
train.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Jet Airways,2019-03-06,Delhi,Cochin,05:30:00,19:45:00,2295,2.0,In-flight meal not included,15864
1,Indigo,2019-05-06,Banglore,Delhi,13:00:00,15:50:00,170,0.0,No Info,3943
2,Multiple Carriers,2019-04-27,Delhi,Cochin,07:10:00,16:10:00,540,1.0,In-flight meal not included,6093
3,Jet Airways,2019-03-21,Delhi,Cochin,18:20:00,18:50:00,1470,2.0,No Info,12604
4,Air India,2019-03-09,Delhi,Cochin,08:00:00,19:15:00,675,1.0,No Info,16754


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640 entries, 0 to 639
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   airline          640 non-null    object 
 1   date_of_journey  640 non-null    object 
 2   source           640 non-null    object 
 3   destination      640 non-null    object 
 4   dep_time         640 non-null    object 
 5   arrival_time     640 non-null    object 
 6   duration         640 non-null    int64  
 7   total_stops      640 non-null    float64
 8   additional_info  640 non-null    object 
 9   price            640 non-null    int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 50.1+ KB


In [6]:
X_train = train.drop(columns="price")
y_train = train.price.copy()

## 4. Transformation Operations

### 4.1 airline

In [7]:
X_train.airline

0            Jet Airways
1                 Indigo
2      Multiple Carriers
3            Jet Airways
4              Air India
             ...        
635            Air India
636          Jet Airways
637          Jet Airways
638             Spicejet
639                Goair
Name: airline, Length: 640, dtype: object

In [8]:
air_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
	("encoder", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))
])

air_transformer.fit_transform(X_train.loc[:, ["airline"]])#.airline.value_counts()

,airline_Air India,airline_Indigo,airline_Jet Airways,airline_Multiple Carriers,airline_Other
0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
635,1.0,0.0,0.0,0.0,0.0
636,0.0,0.0,1.0,0.0,0.0
637,0.0,0.0,1.0,0.0,0.0
638,0.0,0.0,0.0,0.0,1.0


### 4.2 date_of_journey

In [9]:
train.date_of_journey

0      2019-03-06
1      2019-05-06
2      2019-04-27
3      2019-03-21
4      2019-03-09
          ...    
635    2019-03-01
636    2019-06-24
637    2019-03-06
638    2019-04-15
639    2019-06-12
Name: date_of_journey, Length: 640, dtype: object

In [10]:
feature_to_extract = ["month", "week", "day_of_week", "day_of_year"]

doj_transformer = Pipeline(steps=[
	("dt", DatetimeFeatures(features_to_extract=feature_to_extract, yearfirst=True, format="mixed")),
	("scaler", MinMaxScaler())
])

doj_transformer.fit_transform(X_train.loc[:, ["date_of_journey"]])

,date_of_journey_month,date_of_journey_week,date_of_journey_day_of_week,date_of_journey_day_of_year
0,0.000000,0.058824,0.333333,0.042373
1,0.666667,0.588235,0.000000,0.559322
2,0.333333,0.470588,0.833333,0.483051
3,0.000000,0.176471,0.500000,0.169492
4,0.000000,0.058824,0.833333,0.067797
...,...,...,...,...
635,0.000000,0.000000,0.666667,0.000000
636,1.000000,1.000000,0.000000,0.974576
637,0.000000,0.058824,0.333333,0.042373
638,0.333333,0.411765,0.000000,0.381356


### 4.3 source & destination

In [13]:
X_train.source

0         Delhi
1      Banglore
2         Delhi
3         Delhi
4         Delhi
         ...   
635    Banglore
636       Delhi
637       Delhi
638     Kolkata
639       Delhi
Name: source, Length: 640, dtype: object

In [14]:
X_train.destination

0         Cochin
1          Delhi
2         Cochin
3         Cochin
4         Cochin
         ...    
635    New Delhi
636       Cochin
637       Cochin
638     Banglore
639       Cochin
Name: destination, Length: 640, dtype: object

In [15]:
location_subset = X_train.loc[:, ["source", "destination"]]
location_subset

,source,destination
0,Delhi,Cochin
1,Banglore,Delhi
2,Delhi,Cochin
3,Delhi,Cochin
4,Delhi,Cochin
...,...,...
635,Banglore,New Delhi
636,Delhi,Cochin
637,Delhi,Cochin
638,Kolkata,Banglore


In [16]:
location_pipe1 = Pipeline(steps=[
	("grouper", RareLabelEncoder(tol=0.1, replace_with="Other", n_categories=2)),
	("encoder", MeanEncoder()),
	("scaler", PowerTransformer())
])

location_pipe1.fit_transform(location_subset, y_train)

,source,destination
0,1.064408,1.065574
1,-0.788452,-1.867838
2,1.064408,1.065574
3,1.064408,1.065574
4,1.064408,1.065574
...,...,...
635,-0.788452,-0.833325
636,1.064408,1.065574
637,1.064408,1.065574
638,-0.323978,-0.310733


In [17]:
np.union1d(
	X_train.source.unique(),
	X_train.destination.unique()
)

array(['Banglore', 'Chennai', 'Cochin', 'Delhi', 'Hyderabad', 'Kolkata',
       'Mumbai', 'New Delhi'], dtype=object)

In [18]:
def is_north(X):
	columns = X.columns.to_list()
	north_cities = ["Delhi", "Kolkata", "Mumbai", "New Delhi"]
	return (
		X
		.assign(**{
			f"{col}_is_north": X.loc[:, col].isin(north_cities).astype(int)
			for col in columns
		})
		.drop(columns=columns)
	)


FunctionTransformer(func=is_north).fit_transform(location_subset)

,source_is_north,destination_is_north
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
...,...,...
635,0,1
636,1,0
637,1,0
638,1,0


In [19]:
location_transformer = FeatureUnion(transformer_list=[
	("part1", location_pipe1),
	("part2", FunctionTransformer(func=is_north))
])

location_transformer.fit_transform(location_subset, y_train)

,source,destination,source_is_north,destination_is_north
0,1.064408,1.065574,1,0
1,-0.788452,-1.867838,0,1
2,1.064408,1.065574,1,0
3,1.064408,1.065574,1,0
4,1.064408,1.065574,1,0
...,...,...,...,...
635,-0.788452,-0.833325,0,1
636,1.064408,1.065574,1,0
637,1.064408,1.065574,1,0
638,-0.323978,-0.310733,1,0


### 4.4 dep_time & arrival_time

In [20]:
X_train.dep_time

0      05:30:00
1      13:00:00
2      07:10:00
3      18:20:00
4      08:00:00
         ...   
635    08:50:00
636    21:50:00
637    19:45:00
638    17:10:00
639    10:35:00
Name: dep_time, Length: 640, dtype: object

In [21]:
X_train.arrival_time

0      19:45:00
1      15:50:00
2      16:10:00
3      18:50:00
4      19:15:00
         ...   
635    23:25:00
636    19:00:00
637    19:45:00
638    19:40:00
639    19:35:00
Name: arrival_time, Length: 640, dtype: object

In [22]:
time_subset = X_train.loc[:, ["dep_time", "arrival_time"]]
time_subset

,dep_time,arrival_time
0,05:30:00,19:45:00
1,13:00:00,15:50:00
2,07:10:00,16:10:00
3,18:20:00,18:50:00
4,08:00:00,19:15:00
...,...,...
635,08:50:00,23:25:00
636,21:50:00,19:00:00
637,19:45:00,19:45:00
638,17:10:00,19:40:00


In [23]:
time_pipe1 = Pipeline(steps=[
	("dt", DatetimeFeatures(features_to_extract=["hour", "minute"])),
	("scaler", MinMaxScaler())
])

time_pipe1.fit_transform(time_subset)

,dep_time_hour,dep_time_minute,arrival_time_hour,arrival_time_minute
0,0.217391,0.545455,0.826087,0.818182
1,0.565217,0.000000,0.652174,0.909091
2,0.304348,0.181818,0.695652,0.181818
3,0.782609,0.363636,0.782609,0.909091
4,0.347826,0.000000,0.826087,0.272727
...,...,...,...,...
635,0.347826,0.909091,1.000000,0.454545
636,0.913043,0.909091,0.826087,0.000000
637,0.826087,0.818182,0.826087,0.818182
638,0.739130,0.181818,0.826087,0.727273


In [24]:
def part_of_day(X, morning=4, noon=12, eve=16, night=20):
	columns = X.columns.to_list()
	X_temp = X.assign(**{
		col: pd.to_datetime(X.loc[:, col]).dt.hour
		for col in columns
	})

	return (
		X_temp
		.assign(**{
			f"{col}_part_of_day": np.select(
				[X_temp.loc[:, col].between(morning, noon, inclusive="left"),
				 X_temp.loc[:, col].between(noon, eve, inclusive="left"),
				 X_temp.loc[:, col].between(eve, night, inclusive="left")],
				["morning", "afternoon", "evening"],
				default="night"
			)
			for col in columns
		})
		.drop(columns=columns)
	)

FunctionTransformer(func=part_of_day).fit_transform(time_subset)

,dep_time_part_of_day,arrival_time_part_of_day
0,morning,evening
1,afternoon,afternoon
2,morning,evening
3,evening,evening
4,morning,evening
...,...,...
635,morning,night
636,night,evening
637,evening,evening
638,evening,evening


In [25]:
time_pipe2 = Pipeline(steps=[
	("part", FunctionTransformer(func=part_of_day)),
	("encoder", CountFrequencyEncoder()),
	("scaler", MinMaxScaler())
])

time_pipe2.fit_transform(time_subset)

,dep_time_part_of_day,arrival_time_part_of_day
0,1.000000,0.78125
1,0.000000,0.00000
2,1.000000,0.78125
3,0.160194,0.78125
4,1.000000,0.78125
...,...,...
635,1.000000,1.00000
636,0.218447,0.78125
637,0.160194,0.78125
638,0.160194,0.78125


In [26]:
time_transformer = FeatureUnion(transformer_list=[
	("part1", time_pipe1),
	("part2", time_pipe2)
])

time_transformer.fit_transform(time_subset)

,dep_time_hour,dep_time_minute,arrival_time_hour,arrival_time_minute,dep_time_part_of_day,arrival_time_part_of_day
0,0.217391,0.545455,0.826087,0.818182,1.000000,0.78125
1,0.565217,0.000000,0.652174,0.909091,0.000000,0.00000
2,0.304348,0.181818,0.695652,0.181818,1.000000,0.78125
3,0.782609,0.363636,0.782609,0.909091,0.160194,0.78125
4,0.347826,0.000000,0.826087,0.272727,1.000000,0.78125
...,...,...,...,...,...,...
635,0.347826,0.909091,1.000000,0.454545,1.000000,1.00000
636,0.913043,0.909091,0.826087,0.000000,0.218447,0.78125
637,0.826087,0.818182,0.826087,0.818182,0.160194,0.78125
638,0.739130,0.181818,0.826087,0.727273,0.160194,0.78125


### 4.5 duration

In [27]:
X_train.duration

0      2295
1       170
2       540
3      1470
4       675
       ... 
635     875
636    1270
637    1440
638     150
639     540
Name: duration, Length: 640, dtype: int64

In [28]:
(
	X_train
	.duration
	.quantile([0.25, 0.5, 0.75])
	.values
	.reshape(-1, 1)
	# .shape
)

array([[175. ],
       [512.5],
       [925. ]])

In [29]:
class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
	def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
		self.variables = variables
		self.percentiles = percentiles
		self.gamma = gamma


	def fit(self, X, y=None):
		if not self.variables:
			self.variables = X.select_dtypes(include="number").columns.to_list()

		self.reference_values_ = {
			col: (
				X
				.loc[:, col]
				.quantile(self.percentiles)
				.values
				.reshape(-1, 1)
			)
			for col in self.variables
		}

		return self


	def transform(self, X):
		objects = []
		for col in self.variables:
			columns = [f"{col}_rbf_{int(percentile * 100)}" for percentile in self.percentiles]
			obj = pd.DataFrame(
				data=rbf_kernel(X.loc[:, [col]], Y=self.reference_values_[col], gamma=self.gamma),
				columns=columns
			)
			objects.append(obj)
		return pd.concat(objects, axis=1)

In [30]:
RBFPercentileSimilarity(percentiles=[0.4, 0.8]).fit_transform(X_train)

,duration_rbf_40,duration_rbf_80,total_stops_rbf_40,total_stops_rbf_80
0,0.0,0.0,0.904837,0.904837
1,0.0,0.0,0.904837,0.904837
2,0.0,0.0,1.000000,1.000000
3,0.0,0.0,0.904837,0.904837
4,0.0,0.0,1.000000,1.000000
...,...,...,...,...
635,0.0,0.0,0.904837,0.904837
636,0.0,0.0,1.000000,1.000000
637,0.0,0.0,1.000000,1.000000
638,0.0,0.0,0.904837,0.904837


In [31]:
def duration_category(X, short=180, med=400):
	return (
		X
		.assign(duration_cat=np.select([X.duration.lt(short),
									    X.duration.between(short, med, inclusive="left")],
									   ["short", "medium"],
									   default="long"))
		.drop(columns="duration")
	)

In [32]:
def is_over(X, value=1000):
	return (
		X
		.assign(**{
			f"duration_over_{value}": X.duration.ge(value).astype(int)
		})
		.drop(columns="duration")
	)

In [33]:
duration_pipe1 = Pipeline(steps=[
	("rbf", RBFPercentileSimilarity()),
	("scaler", PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
	("cat", FunctionTransformer(func=duration_category)),
	("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
	("part1", duration_pipe1),
	("part2", duration_pipe2),
	("part3", FunctionTransformer(func=is_over)),
	("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
	("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
	("imputer", SimpleImputer(strategy="median")),
	("union", duration_union)
])

duration_transformer.fit_transform(X_train.loc[:, ["duration"]])

,duration_rbf_25,duration_rbf_50,duration_rbf_75,duration_cat,duration_over_1000,duration
0,-0.350515,-0.167338,-0.11955,2.0,1,2.859719
1,2.817981,-0.167338,-0.11955,0.0,0,-0.931623
2,-0.350515,-0.167338,-0.11955,2.0,0,-0.185455
3,-0.350515,-0.167338,-0.11955,2.0,1,1.690049
4,-0.350515,-0.167338,-0.11955,2.0,0,0.086796
...,...,...,...,...,...,...
635,-0.350515,-0.167338,-0.11955,2.0,0,0.490130
636,-0.350515,-0.167338,-0.11955,2.0,1,1.286715
637,-0.350515,-0.167338,-0.11955,2.0,1,1.629549
638,-0.350515,-0.167338,-0.11955,0.0,0,-0.971957


### $.6 total_stops

In [34]:
X_train.total_stops

0      2.0
1      0.0
2      1.0
3      2.0
4      1.0
      ... 
635    2.0
636    1.0
637    1.0
638    0.0
639    1.0
Name: total_stops, Length: 640, dtype: float64

In [35]:
def is_direct(X):
	return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("", FunctionTransformer(func=is_direct))
])

total_stops_transformer.fit_transform(X_train.loc[:, ["total_stops"]])

,total_stops,is_direct_flight
0,2.0,0
1,0.0,1
2,1.0,0
3,2.0,0
4,1.0,0
...,...,...
635,2.0,0
636,1.0,0
637,1.0,0
638,0.0,1


### 4.7 additional_info

In [36]:
info_pipe1 = Pipeline(steps=[
	("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
	("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

info_pipe1.fit_transform(X_train.loc[:, ["additional_info"]])

,additional_info_In-flight meal not included,additional_info_No Info,additional_info_Other
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,1.0,0.0
4,0.0,1.0,0.0
...,...,...,...
635,0.0,0.0,1.0
636,1.0,0.0,0.0
637,0.0,1.0,0.0
638,0.0,0.0,1.0


In [37]:
def have_info(X):
	return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

In [38]:
info_union = FeatureUnion(transformer_list=[
	("part1", info_pipe1),
	("part2", FunctionTransformer(func=have_info))
])

In [39]:
info_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
	("union", info_union)
])

info_transformer.fit_transform(X_train.loc[:, ["additional_info"]])

,additional_info_In-flight meal not included,additional_info_No Info,additional_info_Other,additional_info
0,1.0,0.0,0.0,1
1,0.0,1.0,0.0,0
2,1.0,0.0,0.0,1
3,0.0,1.0,0.0,0
4,0.0,1.0,0.0,0
...,...,...,...,...
635,0.0,0.0,1.0,1
636,1.0,0.0,0.0,1
637,0.0,1.0,0.0,0
638,0.0,0.0,1.0,1


### 4.8 Column Transformer

In [40]:
column_transformer = ColumnTransformer(transformers=[
	("air", air_transformer, ["airline"]),
	("doj", doj_transformer, ["date_of_journey"]),
	("location", location_transformer, ["source", 'destination']),
	("time", time_transformer, ["dep_time", "arrival_time"]),
	("dur", duration_transformer, ["duration"]),
	("stops", total_stops_transformer, ["total_stops"]),
	("info", info_transformer, ["additional_info"])
], remainder="passthrough")

column_transformer.fit_transform(X_train, y_train)

,air__airline_Air India,air__airline_Indigo,air__airline_Jet Airways,air__airline_Multiple Carriers,air__airline_Other,doj__date_of_journey_month,doj__date_of_journey_week,doj__date_of_journey_day_of_week,doj__date_of_journey_day_of_year,location__source,location__destination,location__source_is_north,location__destination_is_north,time__dep_time_hour,time__dep_time_minute,time__arrival_time_hour,time__arrival_time_minute,time__dep_time_part_of_day,time__arrival_time_part_of_day,dur__duration_rbf_25,dur__duration_rbf_50,dur__duration_rbf_75,dur__duration_cat,dur__duration_over_1000,dur__duration,stops__total_stops,stops__is_direct_flight,info__additional_info_In-flight meal not included,info__additional_info_No Info,info__additional_info_Other,info__additional_info
0,0.0,0.0,1.0,0.0,0.0,0.000000,0.058824,0.333333,0.042373,1.064408,1.065574,1,0,0.217391,0.545455,0.826087,0.818182,1.000000,0.78125,-0.350515,-0.167338,-0.11955,2.0,1,2.859719,2.0,0,1.0,0.0,0.0,1
1,0.0,1.0,0.0,0.0,0.0,0.666667,0.588235,0.000000,0.559322,-0.788452,-1.867838,0,1,0.565217,0.000000,0.652174,0.909091,0.000000,0.00000,2.817981,-0.167338,-0.11955,0.0,0,-0.931623,0.0,1,0.0,1.0,0.0,0
2,0.0,0.0,0.0,1.0,0.0,0.333333,0.470588,0.833333,0.483051,1.064408,1.065574,1,0,0.304348,0.181818,0.695652,0.181818,1.000000,0.78125,-0.350515,-0.167338,-0.11955,2.0,0,-0.185455,1.0,0,1.0,0.0,0.0,1
3,0.0,0.0,1.0,0.0,0.0,0.000000,0.176471,0.500000,0.169492,1.064408,1.065574,1,0,0.782609,0.363636,0.782609,0.909091,0.160194,0.78125,-0.350515,-0.167338,-0.11955,2.0,1,1.690049,2.0,0,0.0,1.0,0.0,0
4,1.0,0.0,0.0,0.0,0.0,0.000000,0.058824,0.833333,0.067797,1.064408,1.065574,1,0,0.347826,0.000000,0.826087,0.272727,1.000000,0.78125,-0.350515,-0.167338,-0.11955,2.0,0,0.086796,1.0,0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.666667,0.000000,-0.788452,-0.833325,0,1,0.347826,0.909091,1.000000,0.454545,1.000000,1.00000,-0.350515,-0.167338,-0.11955,2.0,0,0.490130,2.0,0,0.0,0.0,1.0,1
636,0.0,0.0,1.0,0.0,0.0,1.000000,1.000000,0.000000,0.974576,1.064408,1.065574,1,0,0.913043,0.909091,0.826087,0.000000,0.218447,0.78125,-0.350515,-0.167338,-0.11955,2.0,1,1.286715,1.0,0,1.0,0.0,0.0,1
637,0.0,0.0,1.0,0.0,0.0,0.000000,0.058824,0.333333,0.042373,1.064408,1.065574,1,0,0.826087,0.818182,0.826087,0.818182,0.160194,0.78125,-0.350515,-0.167338,-0.11955,2.0,1,1.629549,1.0,0,0.0,1.0,0.0,0
638,0.0,0.0,0.0,0.0,1.0,0.333333,0.411765,0.000000,0.381356,-0.323978,-0.310733,1,0,0.739130,0.181818,0.826087,0.727273,0.160194,0.78125,-0.350515,-0.167338,-0.11955,0.0,0,-0.971957,0.0,1,0.0,0.0,1.0,1
